### PGE Driver
This notebook is the driver by which the hello-world notebook will be processed in order to be available as a HySDS PGE. The notebook *hello_world_submit_job* is then used to submit it to HySDS for execution.


#### 1. Set-up
To start, we; enter the directory containing the notebook to be executed, ensure that we are in the proper branch, then execute notebook-pge-wrapper to generate the files necessary to build and run the *papermill*'ed notebook as a PGE.

In [1]:
%%bash
cd ~/hello_world
git checkout develop
notebook-pge-wrapper specs all

M	hello_world_pge_create.ipynb
M	hello_world_submit_job.ipynb
M	notebook_pges/hello_world_notebook.ipynb
Your branch is up to date with 'origin/develop'.
.ipynb_checkpoints is not a notebook, skipping...
inspecting notebook: hello_world_notebook.ipynb
generated docker/hysds-io.json.hello_world_notebook
generated docker/job-spec.json.hello_world_notebook


Already on 'develop'
2021-02-03 16:16:27 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb
2021-02-03 16:16:27 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb
2021-02-03 16:16:27 [INFO] Input Notebook:  notebook_pges/hello_world_notebook.ipynb


### Important Note.
A manual edit to the job-spec file is necessary. In the line:

>`    "command": "notebook-pge-wrapper execute /home/ops/hello_world/notebook_pges/hello_world_notebook.ipynb",`
 
Change *'ops'* to *'jovyan'*:

>`    "command": "notebook-pge-wrapper execute /home/jovyan/hello_world/notebook_pges/hello_world_notebook.ipynb",`
 

#### 2. Commit changes
The notebook as a PGE is obtained by the system via a git checkout, in this case from github, so we need to make sure the remote repository is up to date. There may not be anything to commit, but we need to ensure consistency - there is no harm in trying to commit in the absence of changes.

In [2]:
%%bash
cd ~/hello_world
git add -A
git commit -m"Results of notebook-pge-wrapper spec generation."
git push

[develop ed58e85] Results of notebook-pge-wrapper spec generation.
 4 files changed, 75 insertions(+), 96 deletions(-)


To https://github.com/jimmie/hello_world
   6b22ad3..ed58e85  develop -> develop


#### 3. Initialize the api that will be used to specify the PGE job to HySDS

In [3]:
import os
from pathlib import Path
import otello
if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

#### 4. Specify the repository and branch to the api
This creates a handle to the job specification.

In [4]:
repository = "https://github.com/jimmie/hello_world.git"
branch = "develop"

ci = otello.CI(repo=repository, branch=branch)

#### 5. Ensure the most current version of the job is registered
If the job exists previously, unregister it. Regardless, register the job to ensure the most current version. 

In [7]:
try:
    if (ci.check_job_exists()):
        print("Job exists, unregistering")
        ci.unregister()
except Exception as e:
    print("Exception : {}, trying to register anyway".format(e))

ci.register()

Job exists, unregistering
[2021-02-03 16:28:50,058: DEBUG/sdscli/ci] sds_type: hysds
[2021-02-03 16:28:50,061: DEBUG/sdscli/__init__] file: /export/home/hysdsops/.sds/config
[2021-02-03 16:28:50,425: DEBUG/sdscli/get_func] mod: <module 'sdscli.adapters.hysds.ci' from '/export/home/hysdsops/mozart/ops/sdscli-1.0.6/sdscli/adapters/hysds/ci.py'>
[2021-02-03 16:28:50,426: DEBUG/sdscli/get_func] func_name: add_job
[2021-02-03 16:28:50,426: DEBUG/sdscli/ci] func: <function add_job at 0x7f5c8786d5e0>
[2021-02-03 16:28:50,426: DEBUG/sdscli/__init__] file: /export/home/hysdsops/.sds/config
[100.64.122.201] Executing task 'add_ci_job'
[100.64.122.201] run: mkdir -p /var/lib/jenkins/jobs/nisar-jdyoung-21_container-builder_jimmie_hello_world_develop
[100.64.122.201] run: chmod -R 777 /var/lib/jenkins/jobs/nisar-jdyoung-21_container-builder_jimmie_hello_world_develop
[100.64.122.201] put: <file obj> -> tmp-jenkins-upload
[100.64.122.201] run: cp -rp tmp-jenkins-upload /var/lib/jenkins/jobs/nisar-jd

#### 6. Build the PGE

In [8]:
ci.submit_build()

{'success': True,
 'message': 'job successfully submitted to build nisar-jdyoung-21_container-builder_jimmie_hello_world_develop (1)',
 'queue': 50055,
 'build_number': 1}

#### 7. Wait for the build to complete, then sample the result.

In [9]:
import time
move_on = False
while not move_on:
    try:
        ci.get_build_status()
        move_on = True
    except Exception as e:
        print("Waiting for build to start")
        time.sleep(10)
    
build_stat = ci.get_build_status()
print("Build in progress.", end="")

while (build_stat['building']):
    time.sleep(10)
    print(".", end="", flush=True)
    build_stat = ci.get_build_status()

if (build_stat['result'] == 'FAILURE'):
    print("\nBuild Failed.")
else:
    print("\nBuild completed successfully")

Waiting for build to start
Build in progress.......................................................................
Build completed successfully


#### 8. Get the final details on the build completion.

In [ ]:
build_stat = ci.get_build_status()
print(build_stat)
if (build_stat['result'] == 'FAILURE'):
    print("Build Faild.")
else:
    print("Build completed successfully")